In [6]:
import pandas as pd
import numpy as np
import re

In [276]:
import os
path = "주가총액csv"
ju_file_list = os.listdir(path)
ju_file_list

['2011.csv',
 '2012.csv',
 '2013.csv',
 '2014.csv',
 '2015.csv',
 '2016.csv',
 '2017.csv',
 '2018.csv',
 '2019.csv',
 '2020.csv',
 '2021.csv']

In [294]:
for f in ju_file_list[:1]:
    df = pd.read_csv(f"./주가총액csv/{f}", encoding='euc-kr')
df[['종목명', '종가', '시가총액', '상장주식수']]
df['년도'] = 2011
df

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,년도
0,069110,3H,KOSDAQ,NaN,4245,295,7.47,4000,4430,4000,384647,1626751425,57161200320,13465536,2011
1,060310,3S,KOSDAQ,NaN,4725,-60,-1.25,4785,4850,4650,154875,733006765,154196576100,32634196,2011
2,900010,3노드디지탈,KOSDAQ,NaN,1790,-40,-2.19,1835,1860,1765,2626663,4742980805,103612055700,57883830,2011
3,038120,AD모터스,KOSDAQ,NaN,563,6,1.08,567,575,554,2019338,1133497855,48912920351,86879077,2011
4,013340,AJS,KOSDAQ,NaN,1820,-10,-0.55,1840,1870,1765,5321067,9751550195,65535024100,36008255,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967,000545,흥국화재우,KOSPI,NaN,2245,25,1.13,2220,2290,2210,2468,5501810,1724160000,768000,2011
1968,003280,흥아해운,KOSPI,NaN,1150,5,0.44,1160,1160,1140,251380,288362250,81320444500,70713430,2011
1969,037440,희림,KOSDAQ,NaN,7900,80,1.02,7870,7970,7830,35005,276457560,109987552500,13922475,2011
1970,123160,히든챔피언스팩1호,KOSDAQ,NaN,1930,10,0.52,1950,1950,1930,1707,3294530,31073000000,16100000,2011


In [ ]:
path = "./fsdata"
file_list = os.listdir(path)
file_list

In [260]:
def concat_df(df, name, name_pos):
    col_list = []
    for i, col in enumerate(df.columns):
        try:
            int(col[:8])
            col_list.append(i)
        except:
            pass

    nm_df = df.iloc[:, name_pos]
    ye_df =  df.iloc[:, col_list]
    con_df = pd.concat([nm_df, ye_df], axis=1)
    filter_col = ['name']
    for col in con_df.columns[1:]:
        filter_col.append(col[:4])
    con_df.columns = filter_col
    con_df.name.iloc[0] = name
    
    con_df.set_index('name', inplace=True)

    con_df = con_df.T.reset_index().drop_duplicates(subset=['index']).set_index('index').T
    con_df.columns.name = ''
    return con_df

In [275]:
df_li = []
count = 0
error_li = []
for i, f in enumerate(file_list[:10]):
   
        df = pd.read_excel(f"./fsdata/{f}", sheet_name=None)

        if df['Data_cis'].iloc[:, 1][0] == "label_ko":
            label_col = 1
        else:
            label_col = 2

        if 'Data_is' in df.keys():
            sheet_name = 'Data_is'
        else:
            sheet_name = 'Data_cis'

        # 자본금   
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].str.contains('자본금').fillna(False)] 
        if len(df1) > 1:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자본금']
        elif len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].str.contains('ifrs-full_IssuedCapital').fillna(False)] 
            if len(df1) == 0:
                df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].isin(['ifrs_EquityAttributableToOwnersOfParent']).fillna(False)] 
        if len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].isin(['납입자본'])] 

        # 자산총계
        df2 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자산총계']

        # 당기순이익
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기순이익').fillna(False)]
        if len(df3) > 1:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['연결당기순이익', '당기순이익(손실)', '당기순이익', '당기순손익', '당기순손실'])]
            if len(df3) == 0:
                df3 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].isin(['당기순이익(손실)'])]
        elif len(df3) == 0:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['당기순손익', '당기순손실', '당기의 순이익', '계속영업이익(손실)'])]
            if len(df3) == 0:
                df3 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('당기순이익').fillna(False)]
            if len(df3) == 0:
                df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기손익').fillna(False)]
        if len(df3) > 1:
            df3 = df3[df3.isna().sum(axis = 1) == df3.isna().sum(axis = 1).min()]

        # 영업활동 현금흐름
        regex = r'[1-9]?[A-z]?[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]?\s?영업\s?활동?([가-힣]+)?(\s+[가-힣]+\s+)?\s?현금\s?흐름'
        df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains('ifrs-full_CashFlowsFromUsedInOperatingActivities').fillna(False)]                          

        if len(df4) == 0:
            df4 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].str.contains(regex).fillna(False)]
            if len(df4) > 1:
                df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].isin(['ifrs_CashFlowsFromUsedInOperatingActivities']).fillna(False)]
                if len(df4) > 0:
                    df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains(r'영업\s활동\s현금\s흐름').fillna(False)]

        # 영업이익
        df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('dart_OperatingIncomeLoss').fillna(False)]
        if len(df5) == 0:
            df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
            if len(df5) == 0:
                df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains(r'영업\s?이익').fillna(False)]
                if len(df5) == 0:
                    sheet_name = 'Data_cis'
                    df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]

        # 매출액
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('ifrs-full_Revenue').fillna(False)] 
        if len(df6) == 0: # ID로 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        if len(df6) == 0: # Name으로 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('매출액').fillna(False)]
            if len(df6) == 0:
                df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].isin(['매출'])]

            if len(df6) > 1: 
                df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'Ⅰ.매출액.*').fillna(False)]


        if len(df6) == 0: # name col다른 애들 col변경해서 잡기

            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('매출액').fillna(False)]
            if len(df6) > 1:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['매출액'])]
            if len(df6) == 0:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.매출'])]
            if len(df6) == 0:
                df6 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('매출액').fillna(False)]

        if len(df6) == 0:   # 매출액 없는녀석 영업수익 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('영업수익').fillna(False)]
            if len(df6) > 1: # name으로 isin
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
                if len(df6) > 1:
                    df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
        elif len(df6) > 1:
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        
        df1 = concat_df(df1, '자본총계', label_col)
        df2 = concat_df(df2, '자산총계', label_col)
        df3 = concat_df(df3, '당기순이익', label_col)
        df4 = concat_df(df4, '영업활동현금흐름', label_col)
        df5 = concat_df(df5, '영업이익', label_col)
        df6 = concat_df(df6, '매출액', label_col)
        con_df = pd.concat([df1, df2, df3, df4, df5, df6])
        
        file_n = f.replace('.xlsx', '.csv')
        con_df.to_csv(f'./data/{file_n}')

        print(i, f, len(df1), len(df2), len(df3), len(df4), len(df5), len(df6))
#         if len(df1) != 1 or len(df2) != 1 or len(df3) != 1 or len(df4) != 1 or len(df5) != 1 or len(df6) != 1:
#             count += 1
#             error_li.append([i, f])
#             print('Error')
    
print(error_li)
print(count)

0 3S.xlsx 1 1 1 1 1 1
1 AJ네트웍스.xlsx 1 1 1 1 1 1
2 AK홀딩스.xlsx 1 1 1 1 1 1
3 APS홀딩스.xlsx 1 1 1 1 1 1
4 AP시스템.xlsx 1 1 1 1 1 1
5 BGF.xlsx 1 1 1 1 1 1
6 BGF리테일.xlsx 1 1 1 1 1 1
7 BNGT.xlsx 1 1 1 1 1 1
8 BNK금융지주.xlsx 1 1 1 1 1 1
9 BYC.xlsx 1 1 1 1 1 1
[]
0
